Libraries

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('wordnet')
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jayso\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df_mental_health = pd.read_csv('../datasets/datasets.csv')

### Text pre-processing

In [3]:
# text preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text, stop_words):
    if text and isinstance(text, str):
        text = BeautifulSoup(text, "html.parser").get_text()
        text = re.sub(r'https?://\S+|www\.\S+|@\w+|#\w+|[^a-zA-Z]', ' ', text.lower())
        text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if len(word) > 1 and word not in stop_words])
        text = ' '.join(list(dict.fromkeys(text.split())))
    else:
        text = ''
    return text

df_mental_health['clean_text'] = df_mental_health['text'].apply(clean_text, stop_words=stop_words)
df_mental_health[['text', 'clean_text']].sample(10)

,text,clean_text
2852,"Hello everyone. I am feeling like I am going backward and not doing anything right. \n\nI am 26 years old, been in college for 3 years but it still take 4 more years if I dont fall any class. My gpa is 1.6 on probabtion. I finally found out my school has free psychologist for student. Hopefully I will get diagnosed and get some help. I am dead broke. I have 0.96 cent in my bank account. I work 20 hours a week but still suck at school. I live by myself , my parents are not in the US. I am feeling like a failure, have been worst student in class all my life. I appreciate education a lot, but I feel like I am wasting my family money... My older brother is gonna get his Master next year. Probably wont get the bachelor at 30 years old. A lot of my young relative is done or almost done with their bachelor back in my country. \n\nMost of classmate are younger than me and a lot of people in this forum. I hurt me everytime I see people get diagnosed, medicated early. I know I cant blame on that. Its just me... I feel so stupid, my dream is to be a doctor...ironically. I just start crying writing this. \n\nI wonder how are you guys doing. \n\n",hello everyone feeling like going backward anything right year old college still take dont fall class gpa probabtion finally found school free psychologist student hopefully get diagnosed help dead broke cent bank account work hour week suck live parent u failure worst life appreciate education lot feel wasting family money older brother gonna master next probably wont bachelor young relative done almost back country classmate younger people forum hurt everytime see medicated early know cant blame stupid dream doctor ironically start cry writing wonder guy
11650,"After years, IÃ¢â‚¬â„¢ve mastered the art of waking up early, with an app called Timely. However, I find it impossibly to get to bed early. I know very well there are consequences if I sleep late. I can barely function properly with less than 8 hours of sleep and my acne worsens. But when it is time to get ready to sleep, my brain constantly chooses the instant gratification of listening to one more song or reading one more post.",year mastered art waking early app called timely however find impossibly get bed know well consequence sleep late barely function properly le hour acne worsens time ready brain constantly chooses instant gratification listening one song reading post
4966,"Like the title says, it's unlikely that I'll be able to get diagnosed anytime soon, because my life isn't severely impacted enough to justify the cost to my parents. I relate to pretty much every comment I see on this sub, and I have a good majority of the symptoms described online. I'm like 95% sure I have adhd.\n\nBecause I won't be getting an appointment anytime soon, I can't get any medication. (Even if I did, I'd probably never remember to go to the pharmacy during operating hours in order to pick it up, lol) Is there any over the counter solution? Also, any general lifestyle changes that you guys would seriously recommend?",like title say unlikely able get diagnosed anytime soon life severely impacted enough justify cost parent relate pretty much every comment see sub good majority symptom described online sure adhd getting appointment medication even probably never remember go pharmacy operating hour order pick lol counter solution also general lifestyle change guy would seriously recommend
6240,"Successfully overcame anxiety today! Hello all! Been lurking on this sub for a while, but I was feeling on top of the world today and wanted to share!\n\nMy wife and I got a letter in the mail yesterday that our state taxes from 2016 were audited, and we owed a bunch more money. Instantly my anxiety kicked in and I could feel my blood pressure rising.\n\nMy wife tasked me with reaching out to the folks who did our taxes that year to argue with them that since they messed up we shouldn't have to pay. I'm AWFUL at confrontation and conflict

Sentiment Labeling with TEXTBLOB

In [4]:
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

df_mental_health['textblob_polarity'] = df_mental_health['clean_text'].apply(get_sentiment).round(2)

def categorize_sentiment(score):
    if score >= 0.5:
        return 'Positive'
    elif score >= 0.05 and score < 0.5:
        return 'Moderately Positive'
    elif score > -0.05 and score < 0.05:
        return 'Neutral'
    elif score > -0.5 and score <= -0.05:
        return 'Moderately Negative'
    else:
        return 'Negative'
    
df_mental_health['sentiment_textblob'] = df_mental_health['textblob_polarity'].apply(categorize_sentiment)
df_mental_health[['clean_text', 'textblob_polarity', 'sentiment_textblob']].sample(10)

,clean_text,textblob_polarity,sentiment_textblob
3065,diagnosed adhd last year struggle figure career led unhappiness work industry believe ultra conservative harm world good progressed technician engineer assistant project manager make money feeling pigeon holed every day performance le stellar mainly quality low interest struggling change always seems kick squash motivation follow daydreamed finding perfect fit someone like flourish could find go hit serotonin produced thought loving everyday bout depression never happen described race horse whose gate open hell driven nothing show relate specific job suited adult commit direction varies wildly,0.24,Moderately Positive
7766,setting music could homework force habit started scroll comment second said temporary success forever believe brief moment consideration felt like snapped something immediately switched tab feeling got afterwords high happen often make everything seem possible,0.09,Moderately Positive
3016,using alarm clock app called barcode scan item set deleted changed go order turn keep kitchen get bed annoying pas realize actually,-0.40,Moderately Negative
359,every time specific coworker sends work related mail ask something panick treated badly past severe anxiety lead procrastination subpar performance fix sort pavlovian response make feel sick colleague terrible person per se put bad spot group everyone calling technically even fault ended scapegoat bos done whenever writes report stuff absolutely open email fear asking yet want reprimand obviously procrastinating late responding result giving subpart project man without constant wrong fired fight transform leading back normal approachable think feeling stomach know judgemental yes become monster eye let criticism tapped insecurity go able job,-0.19,Moderately Negative
3620,worst thing anxiety going tough time would hard anyone people think problem head due know reacting differently mean everything handle also dealt stuff making overwhelmed maybe overreact another stressor,-0.42,Moderately Negative
1542,girlfriend problem needing space online avpd avoidant personality disorder today asked first time ever dating month almost little surprised usually needy sure bad day within past never also understand good thing get point asking experience however sign going somewhat downhill week disrespected ignored put horrible anxiety cheered made feel better bit later wanted watch show frequently said needed alone apologized told need relationship way last start preparing breakup,-0.02,Neutral
2174,even working info make reddit twitter whatever else daily distraction seem pointless head,-0.12,Moderately Negative
6895,hour first day back school summer project supposed read fahrenheit sister keeper also keep dialectical journal analyze quote stuff completely forgot ap world history beginning adderal starting rate would finished week decided take break around time med causing bit paranoia lot anxiety depression stayed month half doctor appointment told said wait got system two later switched anti depressant effect start appearing point already gone kind still felt unable even focus finishing single page waiting sophomore year le due second walk literally worth work done nothing pray stop,0.03,Neutral
3058,writing woke middle night sweating thinking find write thought constantly fluctuate state feeling extremely smart capable anything complete uselessness live either top world shadow rarely go peacefully knowing fine anxiously realizing prepared slightest jump calmly accepting task never done quickly estimating confidently showing success knew could along hour switch suffering imposter syndrome truly since kid people telling almost flunked college lazy spend countless researching learning new topic internet say going proof dismissed luck timing result deceiving others intelligent competent believe fake well become pretending grade school high get straight honor class even skipped math level multiple time easy reason went private 

### Sentiment Label with Vader

In [5]:
def get_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(text)
    return sentiment_scores['compound']

df_mental_health['vader_polarity'] = df_mental_health['clean_text'].apply(get_sentiment).round(2)

def categorize_sentiment(score):
    if score >= 0.5:
        return 'Positive'
    elif score >= 0.05 and score < 0.5:
        return 'Moderately Positive'
    elif score > -0.05 and score < 0.05:
        return 'Neutral'
    elif score > -0.5 and score <= -0.05:
        return 'Moderately Negative'
    else:
        return 'Negative'

df_mental_health['sentiment_vader'] = df_mental_health['vader_polarity'].apply(categorize_sentiment)
df_mental_health[['clean_text', 'vader_polarity', 'sentiment_vader']].sample(10)

,clean_text,vader_polarity,sentiment_vader
3333,number job year left willingly sure know mean diagnosed med maintained time got better grade school stopped running subsequently missing doctor appointment get refill overlooking scheduling another one longer anymore failed engineering college work luckily still able related past important anyways current really like pay good last position fired consistently late much clocking deadline atmosphere people happier cube open office enviro show anytime within hour window long whole department near age smart funny bottom line want issue lot thing adhd affect performance accomodations feel would help adult probably consider babying holding hand project etc question way tell supervisor struggle bring stigmatization say anything hope fuck big enough let go hired july light september early oct given first real already disappointed progress thus far finally october two week done almost little change back impression make hey sorry disappointing month wasnt treated recently ill eloquently practice id imagine im worried create level doubt ability keep trucking forward try positive concern health coverage plan afford cover medication quite expensive may require extra productive frequent checkup reminder pressure guidance kind explanation warning dont seem excuse regret hiring overthinking idk see guy similar experience,0.96,Positive
8801,henkuyinepu yeah apparently really bad taste,-0.38,Moderately Negative
11512,life general detached person ive always sensitive struggled empathy cared people never simply dont like watching suffer exclusion single changed time theme really care excluding mom dad sister except one latch onto take long attachment form becomes everything theyre think want ever get annoyed bored company unlike every question adhd related issue something else recently reading seen u hyper exact opposite havent anything thats combination sort tldr emotional detachment exception find someone become entirely sorry formatted poorly putting wrong thread,0.15,Moderately Positive
10765,happening let start saying idea right place write talk please need help male turned december think going mad maybe stress everything last year hereditary basically feeling anxious today reached peak went hardware store wanted build wood raised garden plan drawn list material needed get loose confidence walked aimlessly isle avoiding people dodging stare got back asked guy question two lumber left still talking felt terrible used confident walk head held high even look eye feel judged constantly really sorry know much worse heart go like could beginning bad spiral confront want better advice anyone give experiencing done calm nerve p incapable presenting front group creating conversation little bit hitting girl bar thank reading end fish,-0.38,Moderately Negative
6678,stricko found pretty frustrating stupid monkey,-0.48,Moderately Negative
3393,first psychologist appointment anyone remember feel like got nothing done know need get everything scheduled hour minute help anxiety anything want come weekly normal,0.59,Positive
7642,getting shit mental health day burnout often anxiety know sustainable call work month get overwhelmed feel like need yesterday one little sister told coworkers talking calling make terrible yet would really set back take hurt minimum wage job nursing home respect big deal wish people understanding minded business,-0.51,Negative
6986,got ebay blue item want jumped bidder hour still ha go better get hope,0.82,Positive
5528,appointment doctor tomorrow probably least general anxiety since always looked past kept truckin along life year gotten bad needed finally reach anything school physical think office last inducing need bring prepared say know work ask med therapy seem like thing given go konw basically going intention saying ruining please help,0.32,Moderately Positive
10142,hi guy find enjoy thing le med day took break concerta watched alien enjoyed ending like mad motherhood theme str

In [6]:
df_mental_health.sample(10)

,text,new_category,clean_text,textblob_polarity,sentiment_textblob,vader_polarity,sentiment_vader
8629,"\n# Let's Celebrate Our Victories\n\nADHD is a daily challenge. Sometimes it's hard to remember the positive and it can feel like things are rarely good. **We win every single day.** We challenge you to write down your wins and see if you feel better looking at the list later in the week. DonÃ¢â‚¬â„¢t worry if you miss a day or two or three! Do what you can. Even writing them down one day is a win.\n\n---\n\n> One thing that comes out in myths is that at the bottom of the abyss comes the voice of salvation. The black moment is the moment when the real message of transformation is going to come. At the darkest moment comes the light. \n> Ã¢â‚¬â€ *Joseph Campbell, The Power of Myth*\n\n\n---\n\n**Examples from previous weeks:**\n\n* Abstaining from binge drinking for 11 days. Keep it up!\n\n* Worked to overcome their traumatic brain injury and had an awesome week.\n\n* I successfully adulted today.\n\n* I just got through 2 weeks of studying and finals and I did pretty good!\n\n* This weekend I was diagnosed with ADD, and people keep telling me they're sorry -- but I'm ecstatic! \n\n* I just finished my bachelors degree.\n\n---\n\n##### We love you, /r/adhd! BE PROUD and celebrate with each other! Ã¢â‚¬â€ your community managers (and /u/blynng)\n\n**Don't forget to join our other exciting weekly threads on [Fridays](https://www.reddit.com/r/ADHD/search?q=flair%3A%27ff%27&sort=new&restrict_sr=on) and [Sundays](http://www.reddit.com/r/ADHD/search?q=flair%3A%27kss%27&sort=new&restrict_sr=on)!**\n",POSITIVE,let celebrate victory adhd daily challenge sometimes hard remember positive feel like thing rarely good win every single day write see better looking list later week worry miss two three even writing one come myth bottom abyss voice salvation black moment real message transformation going darkest light joseph campbell power example previous abstaining binge drinking keep worked overcome traumatic brain injury awesome successfully adulted today got studying final pretty weekend diagnosed add people telling sorry ecstatic finished bachelor degree love proud community manager blynng forget join exciting weekly thread friday sunday,0.26,Moderately Positive,0.96,Positive
8330,"I'm 51 and diag'd w/ADHD a year ago. Doc started me off on what he called ""the Cadillac of ADD meds,"" Vyvanse (12hr). That was alright, but left me no control over my situation - I HAD to be dosed up all day long. It ruined my sleep schedule, and even worse made my digestive tract a stellar mess. So I switched to Adderall 30 IR. I break them up into quarters and take them about 4 hrs apart throughout the day to give me an even and smoother dosing, which allows me to stop taking it about 3-5 hrs before bedtime. If I didn't do that, I'd be up way too late and my schedule would keep shifting off by an hour or two every day.\n\nBut it seems I'm running into a problem...\n\nSome nights, if I have been off the med for a few hours and I'm tired, the unholy dark passenger of rage that has lived inside me since I was brought into this world can explode with untold fury if provoked. Last night I burst into a rage that was near-blackout level, but not quite. I exploded at a beloved family member (also struggling with ADHD but as-of-yet unmedicated) and though he is an adult male, he immediately fell into an intense state of fear due to my explosive rage. The rage was triggered by something he said, but in all honesty, what he said is not the kind of thing that would normally (either on meds, or prior to taking any ADHD meds) upset me like that. \n\nSo I started to wonder, does anyone else have a problem with emotional regulation when your meds wear off? Or am I an anomaly? I have to admit, feeling like I can explode like that without the medication worries me and I am starting to wonder who, or what, I might be if I were to stop? Does taking these meds for too long a

In [7]:
df_mental_health.to_csv('../datasets/data_preprocessed.csv', index=False)